In [ ]:
#------ ASLTransalator 1.0 ---------

In [1]:
# Libraries
%pip install google-genai watchdog

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from google import genai
from google.genai import types
from datetime import datetime, timedelta

In [ ]:
# Gemini API Key
YOUR_API_KEY = ""
client = genai.Client(api_key=YOUR_API_KEY)

In [4]:
# Create Test File for input
input_filename = "input.txt"

with open(input_filename, "w", encoding="utf-8") as f:
    f.write("The duck quacked.\n")
    f.write("We are using the Gemini API for this project.\n")
    f.write("This notebook provides a great environment for prototyping.\n")

print(f"Created file: {input_filename}")

Created file: input.txt


In [5]:
# Configs
model_name = "gemini-2.5-flash"  # Fast and capable model for translation
output_filename = "gemini_output_log.txt"

In [6]:
## Read input 
try:
    with open(input_filename, "r", encoding="utf-8") as f:
        file_content = f.read()
except FileNotFoundError:
    print(f"Error: Input file '{input_filename}' not found.")
    exit()

# 2. Craft the translation prompt
# Using a clear system instruction and user prompt ensures high quality and clean output.
system_instruction = f"You are an expert ASL translator who translates textual and spoken English input into ASL gloss. Do not include any extra commentary, headers, or surrounding text; only provide the translated text."

prompt = f"Translate the following text into ASL gloss:\n\n---\n{file_content}\n---"

print(f"Sending content of '{input_filename}' to Gemini...")

# 3. Call the Gemini API
response = client.models.generate_content(
    model=model_name,
    contents=prompt,
    config={
        "system_instruction": system_instruction
    }
)

# 4. Extract the translated text
translated_text = response.text.strip()

print("Translation received. Writing to file...")

# 5. Write the translated text to the output file
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(translated_text)

print(f"Translation:\n{translated_text}")

Sending content of 'input.txt' to Gemini...
Translation received. Writing to file...
Translation:
DUCK QUACK. THIS PROJECT, WE USE GEMINI API. THIS NOTEBOOK, GOOD ENVIRONMENT PROTOTYPE.


In [ ]:
WATCH_DIRECTORY = r"C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot\InputFiles"
OUTPUT_DIRECTORY = r"C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot\OutputFiles"


TARGET_FILE_NAME = "input.txt" 
OUTPUT_DIRECTORY =  r"C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot\OutputFiles"


def process_text_change(file_path):
    """
    Reads the text content of the modified file and processes it with Gemini.
    """
    print(f"-> Change detected in: {TARGET_FILE_NAME}. Processing...")
    
    try:
        # 1. Read the LATEST content of the file
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip() # .strip() removes leading/trailing whitespace

        if not content:
            print("!! File is empty after modification. Skipping AI call.")
            return

        system_instruction = f"You are an expert ASL translator who translates textual and spoken English input into ASL gloss. Do not include any extra commentary, headers, or surrounding text; only provide the translated text."

        prompt = f"Translate the following text into ASL gloss:\n\n---\n{content}\n---"

        # 3. Call the Gemini API
        response = client.models.generate_content(
            model=model_name,
            contents=prompt,
            config={
                "system_instruction": system_instruction
            }
        )
        # 4. Save the Output to a Log
        output_filename = os.path.join(OUTPUT_DIRECTORY, "gemini_output_log.txt")
        
        # Open in append mode ('a') to keep a history of all changes
        with open(output_filename, 'a', encoding='utf-8') as outfile:
            outfile.write(f"\n{response.text}\n")
            
        print(f"<- Successfully processed and logged output to: {output_filename}")
        
    except Exception as e:
        print(f"!! An error occurred while processing {file_path}: {e}")

# ----------------------------------------------------------------------

class TextChangeHandler(FileSystemEventHandler):
    """
    Custom handler to process the modified file.
    """
    def on_modified(self, event):
        # We only care about file modification events (not directory changes)
        if not event.is_directory:
            file_path = event.src_path
            
            # CRITICAL: Only process the TARGET file
            if os.path.basename(file_path).lower() == TARGET_FILE_NAME.lower():
                # Introduce a small delay (0.5s) to ensure the file save operation is complete
                time.sleep(0.5) 
                process_text_change(file_path)

# --- Main Execution Loop ---
if __name__ == "__main__":
    
    # 🚨 Initial check for the input file
    input_file_path = os.path.join(WATCH_DIRECTORY, TARGET_FILE_NAME)
    if not os.path.exists(input_file_path):
        print(f"!! WARNING: Input file '{TARGET_FILE_NAME}' not found in '{WATCH_DIRECTORY}'.")
        print("!! Please create the file to start monitoring.")

    event_handler = TextChangeHandler()
    observer = Observer()
    
    # Schedule the handler to monitor the directory non-recursively
    observer.schedule(event_handler, WATCH_DIRECTORY, recursive=False)
    
    # Start the observer thread
    observer.start()
    
    print(f"\n*** Watchdog is RUNNING ***")
    print(f"   Monitoring directory: {WATCH_DIRECTORY}")
    print(f"   Target file:          {TARGET_FILE_NAME}")
    
    try:
        # Keep the main thread alive until interrupted
        while True:
            time.sleep(10)
    except KeyboardInterrupt:
        observer.stop()
        print("\nWatchdog stopped by user (Ctrl+C).")
        
    observer.join()


*** Watchdog is RUNNING ***
   Monitoring directory: C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot
   Target file:          input.txt
-> Change detected in: input.txt. Processing...
<- Successfully processed and logged output to: C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot\OutputFiles\gemini_output_log.txt
-> Change detected in: input.txt. Processing...
<- Successfully processed and logged output to: C:\Users\Morgan\Downloads\fall_25\Capstone1\ASL-Robot\OutputFiles\gemini_output_log.txt

Watchdog stopped by user (Ctrl+C).
